In [3]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision
#train test split
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [4]:

path = '/home/donghyun/workbench/results/pngfiles'
csv_path = '/home/donghyun/workbench/results/output_files/MI_annotation.csv'

In [5]:
df = pd.read_csv(csv_path)
#df['Unnamed: 0']
df.iloc[:, 1:12]

,ALMI,AMI,ASMI,ILMI,IMI,IPLMI,IPMI,LMI,PMI,non,ecg_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5
...,...,...,...,...,...,...,...,...,...,...,...
21794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21833
21795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21834
21796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21835
21797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21836


In [25]:
def custom_collate_fn(batch):
    images = []
    labels = []
    for image, label in batch:
        image = transforms.ToTensor()(image)
        images.append(image)
        labels.append(label)
    images = torch.stack(images)
    return images, labels

In [21]:
class ECG_dataset(Dataset):
    def __init__(self, csv_path, path, transform=None):
        self.csv_path = csv_path
        self.path = path
        self.transform = transform
        self.df = pd.read_csv(csv_path)
        self.df.fillna(0)
        self.image_names = self.df['ecg_id']
        #self.labels = self.df.iloc[:, 1:10].values
        self.labels = self.df['non']
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image = Image.open(os.path.join(self.path, str(
            self.image_names[index]).zfill(5)+'.png'))
        image = image.convert('RGB')
        image = image.resize((224, 224))
        if self.transform is not None:
            image = self.transform(image)
        label = self.labels
        #label = F.one_hot(label.to(torch.int64), num_classes=9)

        return image, label


In [24]:
transform = transforms.Compose(
        [
            transforms.RandomRotation(15),
            transforms.ToTensor(),
        ]
    )

dataset = ECG_dataset(csv_path, path, transform=transform)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True,collate_fn=custom_collate_fn)

for batch_idx, (data, target) in enumerate(train_loader):
    print(data.shape)
    print(target.shape)
    break



TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.series.Series'>

In [ ]:
#visualize train test set
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()  
    
dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))
print(labels)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.series.Series'>